# **Preparations**

### **Importing necessary modules**

In [ ]:
import duckdb as ddb
import os

# **Declaring queries**

### **Get mortality rate by state**

In [ ]:
mortality_by_state_sql = """
-- Estimate population, total deaths, and mortality rate (per 1000) by state
WITH population_by_city AS (
	SELECT
		"IBGE_city_code" AS city_code,
		ANY_VALUE("IBGE_state_code") AS state_code,
		ANY_VALUE("state_name") AS state_name,
		ANY_VALUE("population_estimate_2009") AS population
	FROM "Location"
	GROUP BY city_code
),

deaths_by_city AS (
	SELECT 
		"location_code" AS city_code,
		COUNT(DISTINCT "death_id") AS total_deaths             
	FROM "Death"
	GROUP BY city_code
)

SELECT
	ANY_VALUE(pop.state_name) AS "State",
	SUM(pop.population) AS "Estimated_Population",
	SUM(dth.total_deaths) AS "Total_Deaths",
	ROUND(
		CAST((1000 * SUM(dth.total_deaths) / SUM(pop.population)) AS NUMERIC),
		4
	) AS "Mortality_Rate"
FROM population_by_city pop
JOIN deaths_by_city dth ON pop.city_code = dth.city_code
GROUP BY state_code
ORDER BY "Mortality_Rate" DESC
"""

### **Get the most common cause of death by age**

In [ ]:
death_cause_by_age_sql = """
-- Find the most common cause of death (modal cause) for each age
WITH causes_by_age AS (
    SELECT  
        "cause_id" AS cause_id,
		ANY_VALUE("description") AS description,
        ROUND(CAST("deceased_age" AS NUMERIC), 0) AS age,
        COUNT(DISTINCT "death_id") AS death_count
    FROM "Death"
    GROUP BY age, cause_id
),

age_totals_and_max AS (
    SELECT 
        age,
        MAX(death_count) AS max_death_count,
        SUM(death_count) AS total_deaths_for_age
    FROM causes_by_age
    GROUP BY age
)

SELECT  
    cba.age,
    cba."description" AS most_common_cause,
    cba.death_count AS deaths_from_cause,
    atm.total_deaths_for_age,
    ROUND(100.0 * cba.death_count / atm.total_deaths_for_age, 4) AS cause_percentage
FROM causes_by_age cba
JOIN age_totals_and_max atm 
    ON cba.age = atm.age AND cba.death_count = atm.max_death_count
ORDER BY cba.age
"""

### **Get the average age at death (proxy for life expectancy) by city GDP per capita decile**

In [ ]:
life_expectancy_by_gdp_sql = """
-- Analyze death statistics by GDP per capita decile
WITH city_deciles AS (
    SELECT 
        "IBGE_city_code" AS city_code,
		ANY_VALUE("GDP_per_capita") AS gdp_per_capita,
		ANY_VALUE("population_estimate_2009") AS population,
        NTILE(10) OVER (ORDER BY ANY_VALUE("GDP_per_capita")) AS decile
    FROM "Location"
	GROUP BY city_code
),

city_stats_by_decile AS (
    SELECT 
        decile,
        COUNT(*) AS number_of_cities,
        MIN(gdp_per_capita) AS lowest_gdp_per_capita,
        MAX(gdp_per_capita) AS highest_gdp_per_capita,
        SUM(population) AS total_population
    FROM city_deciles
    GROUP BY decile
),

death_stats_by_decile AS (
    SELECT 
        d.decile,
        COUNT(DISTINCT "death_id") AS total_deaths,
        ROUND(CAST(AVG("deceased_age") AS NUMERIC), 4) AS mean_age_of_death,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY "deceased_age") AS median_age_of_death
    FROM "Death" dt
    JOIN city_deciles d ON dt."location_code" = d.city_code
    GROUP BY d.decile
)

SELECT 
    cs.decile AS decile,
    cs.number_of_cities,
    cs.lowest_gdp_per_capita,
    cs.highest_gdp_per_capita,
	--CAST(lowest_gdp_per_capita AS VARCHAR(20)) + ' - ' CAST(highest_gdp_per_capita AS VARCHAR(20))
		--AS gdp_per_capita,
    cs.total_population,
    ds.total_deaths,
    ds.mean_age_of_death,
    ds.median_age_of_death
FROM city_stats_by_decile cs
JOIN death_stats_by_decile ds ON cs.decile = ds.decile
ORDER BY cs.decile
"""

### **Get the top 3 causes of deaths for children under five, grouped by mother's education level**

In [ ]:
cause_by_mothers_education_sql = """
-- Top 3 causes of death (under age 5) by mother's education level
WITH deaths_by_mother_edu AS (
    SELECT  
        "mother_education_level" AS education_level,
		"cause_id" AS cause_id,
		ANY_VALUE("description") AS cause_description,
        ROUND(AVG(CAST("mother_age" AS NUMERIC)), 2) AS mothers_avg_age,
        COUNT(DISTINCT "death_id") AS death_count
    FROM "Death"
	WHERE 
		"deceased_age" < 5.0 AND education_level IN (0, 1, 2, 3, 4, 5)
    GROUP BY education_level, cause_id
),

ranked_causes AS (
    SELECT  
        education_level,
        cause_id,
        death_count,
        RANK() OVER (
            PARTITION BY education_level 
            ORDER BY death_count DESC
        ) AS cause_rank
    FROM deaths_by_mother_edu
),

edu_totals AS (
    SELECT  
        education_level,
        SUM(death_count) AS total_death_count
    FROM deaths_by_mother_edu
    GROUP BY education_level
)



SELECT
    CASE dme.education_level
        WHEN 0 THEN 'None'
        WHEN 1 THEN 'Elementary School'
        WHEN 2 THEN 'Middle School'
        WHEN 3 THEN 'High School'
        WHEN 4 THEN 'Incomplete Higher Education'
        WHEN 5 THEN 'Complete Higher Education'
        ELSE 'Other'
    END AS mothers_education_level,
    dme."cause_id",
    rc.cause_rank,
    dme.cause_description AS cause_of_death,
    dme.mothers_avg_age,
    rc.death_count AS cause_death_count,
    et.total_death_count,
    ROUND(100.0 * rc.death_count / et.total_death_count, 2) AS cause_percentage

FROM deaths_by_mother_edu dme
JOIN ranked_causes rc 
    ON rc.education_level = dme.education_level 
    AND rc.cause_id = dme.cause_id
JOIN edu_totals et 
    ON et.education_level = dme.education_level
WHERE
	rc.cause_rank <= 3
ORDER BY rc.education_level, rc.cause_rank
"""

### **Get the mortality rate grouped by city population per healthcare unit (inverse healthcare unit density)**

In [ ]:
mortality_by_health_unit_density_sql = """
-- Analyze non-external mortality rates grouped by population per health unit
WITH population_per_unit AS (
    SELECT
        "IBGE_city_code" AS city_code,
        ANY_VALUE("population_estimate_2009") AS population,
        COUNT(DISTINCT basic_health_unit_CNES) AS unit_count,
        ANY_VALUE("population_estimate_2009") / COUNT(DISTINCT basic_health_unit_CNES) AS persons_per_unit
    FROM "Location"
    GROUP BY "IBGE_city_code"
),

city_bins AS (
    SELECT
        city_code,
        population,
        unit_count,
        persons_per_unit,
        CASE 
            WHEN persons_per_unit < 1500 THEN 1
            WHEN persons_per_unit < 2000 THEN 2
            WHEN persons_per_unit < 2500 THEN 3
            WHEN persons_per_unit < 3000 THEN 4
            WHEN persons_per_unit < 4000 THEN 5
            WHEN persons_per_unit < 5000 THEN 6
            ELSE 7
        END AS ppu_range
    FROM population_per_unit
),

non_external_deaths AS (
    SELECT
        "location_code" AS city_code,
        COUNT(*) AS non_external_death_count
    FROM "Death"
    WHERE NOT (
        "cause_id" LIKE 'S%' OR
        "cause_id" LIKE 'T%' OR
        "cause_id" LIKE 'V%' OR
        "cause_id" LIKE 'W%' OR
        "cause_id" LIKE 'X%' OR
        "cause_id" LIKE 'Y%'
    )
    GROUP BY "location_code"
)


SELECT
    CASE ppu_range
        WHEN 1 THEN 'Fewer than 1500'
        WHEN 2 THEN '1500-1999'
        WHEN 3 THEN '2000-2499'
        WHEN 4 THEN '2500-2999'
        WHEN 5 THEN '3000-3999'
        WHEN 6 THEN '4000-4999'
        WHEN 7 THEN '5000 or more'
    END AS individuals_per_health_unit,
    COUNT(*) AS number_of_cities,
    SUM(population) AS total_population,
    SUM(unit_count) AS total_units,
    SUM(non_external_death_count) AS total_non_external_deaths,
    ROUND(
        CAST(1000 * SUM(non_external_death_count) / SUM(population) AS NUMERIC),
        4
    ) AS mortality_rate
FROM city_bins cb JOIN
    non_external_deaths ned
    ON cb.city_code =  ned.city_code
GROUP BY ppu_range
ORDER BY ppu_range
"""

In [ ]:
queries = [mortality_by_state_sql,
           death_cause_by_age_sql,
           life_expectancy_by_gdp_sql,
           cause_by_mothers_education_sql,
           mortality_by_health_unit_density_sql]

## **Running queries**

### **Print tables**

In [ ]:
for q in queries:
    with ddb.connect(os.path.abspath('../database.db')) as con:  
        con.sql(q).show(max_rows=200)


### **Save as .csv**

In [ ]:
root = os.path.abspath('../results/')
paths = [root + '/mortality_by_state.csv',
         root + '/death_cause_by_age.csv',
         root + '/life_expectancy_by_gdp.csv',
         root + '/cause_by_mothers_education.csv',
         root + '/mortality_by_health_unit_density.csv']

for q, p in zip(queries, paths):
    with ddb.connect(os.path.abspath('../database.db')) as con:
        query = f"COPY (\n{q}) TO '{p}' (HEADER, DELIMITER ',')"
        con.sql(query)